In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range

from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [2]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [3]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

0.0006099999999999994

In [4]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.0006000000000000001

In [5]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsNonLinear(df)
#     df = ATRFeature(df)
    df = KeltnerChannelsNonLinear(df)
    df = RSINonLinear(df)
    df = MACDNonLinear(df)
    df = BREAKOUT(df)
    df = ichimokuNonLinear(df)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [6]:
df_an = featureExtraction(df_an)

C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_h'].iloc[i] = last_pivot_h
C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_l'].iloc[i] = last_pivot_l


In [7]:
# df_an['SL'][i] = df_an.low[i-1] 
# df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]

# df_an['SL'][i] = df_an.high[i-1] 
# df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]

In [26]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
for i in range(1,len(df_an)): # add label for all of possible signal
    sl = df_an.low[i-1]
    tp = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]
    if sl < df_an.close[i] and df_an.close[i] < tp:
        for j in range(i,len(df_an)):
            if df_an.low[j] <= sl :
                break
            else:
                if(df_an.high[j] > tp):
                    df_an.SIGNAL[i] = BUY
                    break

    sl = df_an.high[i-1]
    tp = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
    if sl > df_an.close[i] and df_an.close[i] > tp:
        for j in range(i,len(df_an)):
            if df_an.high[j] >= sl :
                break
            else:
                if(df_an.low[j] < tp):
                    df_an.SIGNAL[i] = SELL
                    break
                    
                    

C:\Users\meysam\AppData\Local\Temp\ipykernel_13064\788044110.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = SELL
C:\Users\meysam\AppData\Local\Temp\ipykernel_13064\788044110.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = BUY


In [27]:
df_an['SIGNAL'].max()

1

In [28]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'full_range', 'body_lower',
       'body_upper', 'body_bottom_perc', 'body_top_perc', 'body_perc',
       'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'BB_MA_cross', 'BB_UP_touch',
       'BB_LW_touch', 'EMA_cross', 'KeUp_touch', 'KeLo_touch', 'RSI_level',
       'MACDHIST_level', 'pivots_l', 'pivots_h', 'low_perv', 'high_perv',
       'low_next', 'high_next', 'last_pivot_h', 'last_pivot_l', 'close_perv',
       'BREAK_UP', 'BREAK_DOWN', 'Tenkan_sen', 'Kijun_sen', 'Senkou_Span_A',
       'Senkou_Span_B', 'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch', 'Ten_kij_cross',
    

In [29]:

feature_col = [
    'BB_MA_cross', 'BB_UP_touch','BB_LW_touch',
    'EMA_cross', 'KeUp_touch', 'KeLo_touch', 
    'RSI_level',
    'MACDHIST_level',
#      'BREAK_UP', 'BREAK_DOWN',
        'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch',
        'Ten_kij_cross','Cloud_cross',
#         'full_range',# this is added ...
#        'body_lower', 'body_upper', 
#         'body_bottom_perc', 'body_top_perc',
#        'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
#     'mid_point', 'mid_point_prev_2', 'body_size_prev',
#        'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'direction',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [62]:
X = []
Y = [] 
indeces = []
for i in range(4,len(df_an)):
    feature = []
#     feature.append(df_an.pivots_h[i-1])
#     feature.append(df_an.pivots_l[i-1])
#     feature.append(df_an.pivots_h[i-2])
#     feature.append(df_an.pivots_l[i-2])
#     feature.append(df_an.pivots_h[i-3])
#     feature.append(df_an.pivots_l[i-3])
#     feature.append(df_an.pivots_h[i-4])
#     feature.append(df_an.pivots_l[i-4])
    for fc in feature_col:
        feature.append(df_an[fc][i])
        feature.append(df_an[fc][i-1])
        feature.append(df_an[fc][i-2])
        feature.append(df_an[fc][i-3])
        feature.append(df_an[fc][i-4])

    X.append(feature)
    Y.append(df_an.SIGNAL[i])
    indeces.append(i)
        

In [63]:
# for fc in feature_col:
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx]; idx++;")
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx-1]; idx++;")
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx-2]; idx++;")
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx-3]; idx++;")
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx-4]; idx++;")

In [64]:
SPLIT_INDEX = int(0.8*len(X))
X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]

In [65]:
len(feature)

120

In [66]:
clf = MLPClassifier( solver='adam', alpha=1e-3,learning_rate_init=1e-4, # adam
                    hidden_layer_sizes=(120,120,120), 
                    random_state=1,
                    max_iter=1000,
                    verbose=True,warm_start = True,
                    n_iter_no_change=20,
#                     early_stopping = True,
#                     validation_fraction = 0.2 # default is 0.1
                   )

In [67]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 0.96480694
Iteration 2, loss = 0.91978189
Iteration 3, loss = 0.90502031
Iteration 4, loss = 0.89323878
Iteration 5, loss = 0.88459579
Iteration 6, loss = 0.87781645
Iteration 7, loss = 0.87167966
Iteration 8, loss = 0.86696166
Iteration 9, loss = 0.86167384
Iteration 10, loss = 0.85706531
Iteration 11, loss = 0.85253408
Iteration 12, loss = 0.84777354
Iteration 13, loss = 0.84376387
Iteration 14, loss = 0.83953219
Iteration 15, loss = 0.83553364
Iteration 16, loss = 0.83144017
Iteration 17, loss = 0.82734630
Iteration 18, loss = 0.82376965
Iteration 19, loss = 0.81985532
Iteration 20, loss = 0.81627302
Iteration 21, loss = 0.81256334
Iteration 22, loss = 0.80933623
Iteration 23, loss = 0.80610049
Iteration 24, loss = 0.80381748
Iteration 25, loss = 0.80086827
Iteration 26, loss = 0.79713987
Iteration 27, loss = 0.79394866
Iteration 28, loss = 0.79116914
Iteration 29, loss = 0.78831834
Iteration 30, loss = 0.78598182
Iteration 31, loss = 0.78297376
Iteration 32, los

Iteration 253, loss = 0.31367699
Iteration 254, loss = 0.31296202
Iteration 255, loss = 0.31338576
Iteration 256, loss = 0.30939495
Iteration 257, loss = 0.30839272
Iteration 258, loss = 0.30485879
Iteration 259, loss = 0.30576592
Iteration 260, loss = 0.30301169
Iteration 261, loss = 0.30093130
Iteration 262, loss = 0.29960755
Iteration 263, loss = 0.29696896
Iteration 264, loss = 0.29902698
Iteration 265, loss = 0.29488258
Iteration 266, loss = 0.29454283
Iteration 267, loss = 0.29287238
Iteration 268, loss = 0.29111156
Iteration 269, loss = 0.28746492
Iteration 270, loss = 0.28742515
Iteration 271, loss = 0.28662853
Iteration 272, loss = 0.28297140
Iteration 273, loss = 0.28322775
Iteration 274, loss = 0.28019548
Iteration 275, loss = 0.27936939
Iteration 276, loss = 0.27692925
Iteration 277, loss = 0.27708783
Iteration 278, loss = 0.27332542
Iteration 279, loss = 0.27202390
Iteration 280, loss = 0.27274326
Iteration 281, loss = 0.26977983
Iteration 282, loss = 0.26910637
Iteration 

Iteration 502, loss = 0.05080570
Iteration 503, loss = 0.05201155
Iteration 504, loss = 0.04901719
Iteration 505, loss = 0.04932577
Iteration 506, loss = 0.04967566
Iteration 507, loss = 0.04827172
Iteration 508, loss = 0.04872821
Iteration 509, loss = 0.04774915
Iteration 510, loss = 0.04836303
Iteration 511, loss = 0.04722159
Iteration 512, loss = 0.04667521
Iteration 513, loss = 0.04689426
Iteration 514, loss = 0.04463403
Iteration 515, loss = 0.04537586
Iteration 516, loss = 0.04740894
Iteration 517, loss = 0.04495545
Iteration 518, loss = 0.04308514
Iteration 519, loss = 0.04305035
Iteration 520, loss = 0.04296562
Iteration 521, loss = 0.04393163
Iteration 522, loss = 0.04219848
Iteration 523, loss = 0.04236721
Iteration 524, loss = 0.04128280
Iteration 525, loss = 0.04278267
Iteration 526, loss = 0.04083111
Iteration 527, loss = 0.04061301
Iteration 528, loss = 0.04063915
Iteration 529, loss = 0.03889599
Iteration 530, loss = 0.03856867
Iteration 531, loss = 0.03951356
Iteration 

C:\Users\meysam\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning:

Training interrupted by user.



MLPClassifier(alpha=0.001, hidden_layer_sizes=(120, 120, 120),
              learning_rate_init=0.0001, max_iter=1000, n_iter_no_change=20,
              random_state=1, verbose=True, warm_start=True)

In [68]:
# coefs_0 = np.genfromtxt('coefs_0.csv', delimiter=',')
# coefs_1 = np.genfromtxt('coefs_1.csv', delimiter=',')
# coefs_2 = np.genfromtxt('coefs_2.csv', delimiter=',')
# intercepts_0 = np.genfromtxt('intercepts_0.csv', delimiter=',')
# intercepts_1 = np.genfromtxt('intercepts_1.csv', delimiter=',')
# intercepts_2 = np.genfromtxt('intercepts_2.csv', delimiter=',')

In [69]:
y_perdict = clf.predict(X_test)

In [70]:
clf.score(X_train, Y_train)

0.9988073567258804

In [71]:
clf.score(X_test, Y_test)

0.49133818729600803

In [72]:
conf = confusion_matrix(Y_test, y_perdict)
conf

array([[ 237,  455,   83],
       [ 484, 1532,  453],
       [ 119,  432,  188]], dtype=int64)

In [73]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
df_test['SL'] = 0
df_test['TP'] = 0
for i,idx in enumerate(indeces_test):
    if i >1:
        if y_perdict[i] == BUY:
            df_test['SL'][idx] = df_test.low[idx-1] 
            df_test['TP'][idx] = 2*(df_test.close[idx] - df_test.low[idx-1] ) + df_test.close[idx]
            if df_test['SL'][idx] <  df_test.close[idx]:
                df_test.SIGNAL.loc[idx] = y_perdict[i]
        if  y_perdict[i] == SELL:
            df_test['SL'][idx] = df_test.high[idx-1] 
            df_test['TP'][idx] = 2*(df_test.close[idx] - df_test.high[idx-1] ) + df_test.close[idx]
            if df_test['SL'][idx] >  df_test.close[idx]:
                df_test.SIGNAL.loc[idx] = y_perdict[i]
df_test.reset_index(drop=True, inplace=True)

C:\Users\meysam\AppData\Local\Temp\ipykernel_13064\2408145718.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_13064\2408145718.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_13064\2408145718.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_13064\2408145718.py:16: SettingWithCopyWarning:


A

In [74]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [75]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

prepare_data...
run_test...
Result:
win rate:  0.3448275862068966
-1.0    893
 2.0    470
Name: result, dtype: int64


In [76]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio = 1 # abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss =2# abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_13064\2175366404.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])

min balance  17.244890728215957
